## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal(shape=[28*28,100])) # 784->100
        self.W2 = tf.Variable(tf.random.normal(shape=[100,10])) # 100->10
        self.b1 = tf.Variable(tf.random.normal(shape=[100]))
        self.b2 = tf.Variable(tf.random.normal(shape=[10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x,[-1,28*28])
        h1 = tf.matmul(x,self.W1) + self.b1
        h1_relu = tf.nn.relu(h1)
        logits = tf.matmul(h1_relu, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 118.97848 ; accuracy 0.06915
epoch 1 : loss 106.56913 ; accuracy 0.06016667
epoch 2 : loss 98.58754 ; accuracy 0.057666667
epoch 3 : loss 92.89132 ; accuracy 0.05655
epoch 4 : loss 88.447136 ; accuracy 0.05733333
epoch 5 : loss 84.80898 ; accuracy 0.05945
epoch 6 : loss 81.72373 ; accuracy 0.061516665
epoch 7 : loss 79.04425 ; accuracy 0.06413333
epoch 8 : loss 76.672844 ; accuracy 0.06696667
epoch 9 : loss 74.54055 ; accuracy 0.07046667
epoch 10 : loss 72.59937 ; accuracy 0.07436667
epoch 11 : loss 70.812614 ; accuracy 0.07756667
epoch 12 : loss 69.15292 ; accuracy 0.081533335
epoch 13 : loss 67.5991 ; accuracy 0.08523333
epoch 14 : loss 66.13425 ; accuracy 0.08923333
epoch 15 : loss 64.745445 ; accuracy 0.0935
epoch 16 : loss 63.421734 ; accuracy 0.09746667
epoch 17 : loss 62.154182 ; accuracy 0.1018
epoch 18 : loss 60.93714 ; accuracy 0.10598333
epoch 19 : loss 59.764366 ; accuracy 0.11046667
epoch 20 : loss 58.631935 ; accuracy 0.114816666
epoch 21 : loss 57.536808 ;